# Data Collection

In [3]:
from pytube import YouTube
yt = YouTube('https://www.youtube.com/watch?v=U5fX9tbA_Tg')

In [6]:
print(yt.title)
print(yt.thumbnail_url)

Live from Space: James Zooms w/ NASA Astronauts
https://i.ytimg.com/vi/U5fX9tbA_Tg/maxresdefault.jpg


In [7]:
# streams available to downloads
yt.streams.all()

[<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="22" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001F" acodec="mp4a.40.2" progressive="True" type="video">,
 <Stream: itag="137" mime_type="video/mp4" res="1080p" fps="30fps" vcodec="avc1.640028" progressive="False" type="video">,
 <Stream: itag="248" mime_type="video/webm" res="1080p" fps="30fps" vcodec="vp9" progressive="False" type="video">,
 <Stream: itag="136" mime_type="video/mp4" res="720p" fps="30fps" vcodec="avc1.64001f" progressive="False" type="video">,
 <Stream: itag="247" mime_type="video/webm" res="720p" fps="30fps" vcodec="vp9" progressive="False" type="video">,
 <Stream: itag="135" mime_type="video/mp4" res="480p" fps="30fps" vcodec="avc1.4d401f" progressive="False" type="video">,
 <Stream: itag="244" mime_type="video/webm" res="480p" fps="30fps" vcodec="vp9" progressive="False" type="video">,
 <Stream:

In [10]:
stream = yt.streams.first()
print(stream)

<Stream: itag="18" mime_type="video/mp4" res="360p" fps="30fps" vcodec="avc1.42001E" acodec="mp4a.40.2" progressive="True" type="video">


In [11]:
stream.download('youtube/')

'/Users/rramnauth/Desktop/audio-feature-extraction/youtube/Live from Space James Zooms w NASA Astronauts.mp4'

In [13]:
import pandas as pd

videos = pd.read_csv('youtube_videos.csv')

In [14]:
videos.head()

,link,start,end,class,split_length
0,https://www.youtube.com/watch?v=U5fX9tbA_Tg,0:00,0:35,organic,5
1,NaN,0:35,0:39,electronic,5
2,NaN,0:40,0:47,organic,5
3,NaN,0:47,1:10,electronic,5


In [17]:
import csv

with open('youtube_videos.csv', newline='') as csvfile:
    data = list(csv.reader(csvfile))

print(data[0])

['\ufefflink', 'start', 'end', 'class', 'split_length']
